In [ ]:
##Importing most of the libraries we have used in our assignments that we will use in this project

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score

from sklearn import metrics
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from numpy import array
from numpy import argmax


# Read in the CSV file we will be using for this project 
df = pd.read_csv("./AB_NYC_2019.csv")
df[0::10]

## Data analysis

In [ ]:
# Price
df["price"].describe()

In [ ]:
# neighbourhood groups
df['neighbourhood_group'].value_counts()

In [ ]:
# Avg Price in neighbourhood groups
avg_price = df.groupby("neighbourhood_group")["price"].mean()

print(avg_price)

## Encode Non-Numerical Values to 0 and 1. Drop unnecessary columns from DataFrame and replace NaN values with 0.

In [ ]:
# Categorical Features: neighbourhood_group, neighbourhood, room_type
encode_NYC_df = pd.get_dummies(df, columns = ['neighbourhood_group','room_type'])

# Dropping unnecessary features
encode_NYC_df = encode_NYC_df.drop(columns=['id', 'name', 'host_name', 
                      'last_review', 'host_id', 'neighbourhood'])

# Replace NaN with 0
encode_NYC_df['reviews_per_month']=encode_NYC_df['reviews_per_month'].replace(np.nan, 0)
encode_NYC_df['price']=encode_NYC_df['price'].replace(np.nan, 0)

In [ ]:
# Move price to end of list
cols = encode_NYC_df.columns.tolist()
cols.append(cols.pop(cols.index('price')))
encode_NYC_df = encode_NYC_df.reindex(columns = cols)

# Sort by price
encode_NYC_df.sort_values(by='price',ascending=False,inplace=True)

encode_NYC_df.head()

## Split the Dataset

In [ ]:
# Feature_column and encoding
feature_cols = cols
feature_cols.pop(feature_cols.index('price'))
X = encode_NYC_df[feature_cols]
y = encode_NYC_df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Print the first 5 lines
print(X_test)
print('\n')
print(y_test)


# print the size of the traning set:
print(X_train.shape)
print(y_train.shape)

# print the size of the testing set:
print(X_test.shape)
print(y_test.shape)

## Random Forest Classifier

In [ ]:
## Using Random Forest Classifier with parameters: n_estimators = 19, bootstrap = True, random_state=3
my_RandomForest = RandomForestClassifier(n_estimators = 55, bootstrap = True, random_state=3, 
                                         min_samples_split = 3, min_samples_leaf =2)

my_RandomForest.fit(X_train, y_train)

y_predict_rf = my_RandomForest.predict(X_test)

print(y_predict_rf)

# calculate final accuracy
accuracy_rf = accuracy_score(y_test, y_predict_rf)

print("The accuracy is", "{:.1f}".format(accuracy_rf * 100), "%")

## Linear Regression

In [ ]:
# In the following line, "my_LinReg" is instantiated as an "object" of LinearRegression "class"
my_LinReg = LinearRegression()

# Fitting the model to the training dataset
my_LinReg.fit(X_train, y_train)

# Printing Theta0 using attribute "intercept_":
print(my_LinReg.intercept_)

# Printing [Theta1, Theta2, Theta3] using attribute "coef_":
print(my_LinReg.coef_)

In [ ]:
# Predictions on the testing set
predict_price = my_LinReg.predict(X_test)

print(predict_price)


# NOT SURE IF NEEDED
# Calculating "Mean Square Error" (MSE):
# mse = metrics.mean_squared_error(y_test, predict_price)

# Using numpy sqrt function to take the square root and calculate "Root Mean Square Error" (RMSE)
# rmse = np.sqrt(mse)

# print(rmse)

## Logistic Regression - WIP

# KNN

In [ ]:
# Instantiating KNeighborsClassifier with an list of different K values:

k = [3,5,15,25]

for i in k:
    my_knn = KNeighborsClassifier(n_neighbors = i)

    # Training set
    my_knn.fit(X_train, y_train)

    # Testing on the testing set:
    y_predict_knn = my_knn.predict(X_test)

    print(y_predict_knn)

    # find the accuracy of knn:
    accuracy_knn = accuracy_score(y_test, y_predict_knn)
    print("The accuracy using KNN Algorithm with k =",i, "is", "{:.1f}".format(accuracy_knn * 100), "%")
    

# Decision Tree

In [ ]:
# "my_decisiontree" is instantiated as an "object" of DecisionTreeClassifier "class". 
my_decisiontree = DecisionTreeClassifier(random_state=5)

# We can use the method "fit" of the objects "my_decisiontree" along with training dataset and labels to train the model.
my_decisiontree.fit(X_train, y_train)

# Testing on the testing set:
y_predict_dt = my_decisiontree.predict(X_test)

# percent of correct predictions:
accuracy_dt = accuracy_score(y_test, y_predict_dt)

print("The accuracy using Decision Tree is", "{:.1f}".format(accuracy_dt * 100), "%")